In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [3]:
words=open("names.txt","r").read().splitlines()

In [4]:
chars=sorted(list(set("".join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [6]:
n_embd = 10 
n_hidden = 64 

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) 
for p in parameters:
  p.requires_grad = True

4137


In [7]:
def cmp(s,dt,t):
    ex=torch.all(dt==t.grad)
    app=torch.allclose(dt,t.grad)
    maxdiff=(dt-t.grad).abs().max().item()
    print(f"{s:15s} | exact : {ex} | approximate {app}|  and | maxdiff {maxdiff}")

In [8]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [9]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1) 
# Linear layer 1
hprebn = embcat @ W1 + b1 
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) #
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

h = torch.tanh(hpreact) 

#activation
logits = h @ W2 + b2 
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes 
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

#pytorch's implementation of back propagation
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
    t.retain_grad()

loss.backward()
loss

tensor(3.3509, grad_fn=<NegBackward0>)

In [10]:
dlogprobs=torch.zeros_like(logprobs)
dlogprobs[range(n),Yb]=-1/n
dprobs=(1.0/probs) * dlogprobs
dcounts_sum_inv=(counts*dprobs).sum(1,keepdim=True)
dcounts=(counts_sum_inv*dprobs)
dcounts_sum=(-1/counts_sum**2)*dcounts_sum_inv
dcounts+=torch.ones_like(counts)*dcounts_sum  # since the node is used twice
dnorm_logits=counts*dcounts
dlogits=dnorm_logits.clone()  # we are gonna use it later since this node has been used twice
dlogit_maxes=(-dnorm_logits).sum(1,keepdim=True)
dlogits+=F.one_hot(logits.max(1).indices,num_classes=logits.shape[1])*dlogit_maxes
dh=dlogits@W2.T
dw2=h.T@dlogits
db2=dlogits.sum(0)
dhpreact=(1.0-h**2)*dh
dbngain=(bnraw*dhpreact).sum(0,keepdim=True)
dbnraw=(bngain*dhpreact)
dbnbias=dhpreact.sum(0,keepdim=True)
dbndiff=bnvar_inv*dbnraw

dbnvar_inv=(bndiff*dbnraw).sum(0,keepdim=True)
dbnvar=(-0.5*(bnvar+1e-5)**-1.5)*dbnvar_inv
dbndiff2=(1/(n-1))*torch.ones_like(bndiff2)*dbnvar
dbndiff+=2*bndiff*dbndiff2
dhprebn=dbndiff.clone()
dbnmeani=(-dbndiff).sum(0)
dhprebn+=(1.0/n*(torch.ones_like(hprebn)*dbnmeani))
dembcat=dhprebn@W1.T
dW1=embcat.T@dhprebn
db1=dhprebn.sum(0)
demb=dembcat.view(emb.shape)
dC=torch.zeros_like(C)
for i in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        xi=Xb[i,j]
        dC[xi]+=demb[i,j]
        
cmp("logprobs",dlogprobs,logprobs)
cmp("lprobs",dprobs,probs)
cmp("cocunts_sum_inv",dcounts_sum_inv,counts_sum_inv)
cmp("counts_sum",dcounts_sum,counts_sum)
cmp("counts",dcounts,counts)
cmp("normlogits",dnorm_logits,norm_logits)
cmp("logit_maxes",dlogit_maxes,logit_maxes)
cmp("logit",dlogits,logits)
cmp("h",dh,h)
cmp("w2",dw2,W2)
cmp("b2",db2,b2)
cmp("hpreact",dhpreact,hpreact)
cmp("bngain",dbngain,bngain)
cmp("bnbias",dbnbias,bnbias)
cmp("bnraw",dbnraw,bnraw)
cmp("bnvar_inv",dbnvar_inv,bnvar_inv)
cmp("bnvar",dbnvar,bnvar)
cmp("bndiff2",dbndiff2,bndiff2)
cmp("bndiff",dbndiff,bndiff)
cmp("bmeani",dbnmeani,bnmeani)
cmp("hprebn",dhprebn,hprebn)
cmp("embcat",dembcat,embcat)
cmp("w1",dW1,W1)
cmp("b1",db1,b1)
cmp("emb",demb,emb)
cmp("C",dC,C)

logprobs        | exact : True | approximate True|  and | maxdiff 0.0
lprobs          | exact : True | approximate True|  and | maxdiff 0.0
cocunts_sum_inv | exact : True | approximate True|  and | maxdiff 0.0
counts_sum      | exact : True | approximate True|  and | maxdiff 0.0
counts          | exact : True | approximate True|  and | maxdiff 0.0
normlogits      | exact : True | approximate True|  and | maxdiff 0.0
logit_maxes     | exact : True | approximate True|  and | maxdiff 0.0
logit           | exact : True | approximate True|  and | maxdiff 0.0
h               | exact : True | approximate True|  and | maxdiff 0.0
w2              | exact : True | approximate True|  and | maxdiff 0.0
b2              | exact : True | approximate True|  and | maxdiff 0.0
hpreact         | exact : True | approximate True|  and | maxdiff 0.0
bngain          | exact : True | approximate True|  and | maxdiff 0.0
bnbias          | exact : True | approximate True|  and | maxdiff 0.0
bnraw           | ex